In [1]:
import pandas as pd
import json
import csv
import matplotlib.pyplot as plt
import numpy as np
import time
import datetime

def date2ts(date):
    return datetime.datetime.timestamp(datetime.datetime.strptime(date, "%Y/%m/%d %H:%M:%S"))

def ts2date(timestamp):
    return(datetime.datetime.fromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S'))

date2ts('2022/06/12 21:22:5')

1655040125.0

In [16]:
router_info = {
    '20080029c69feeb': {'lat': 22.3482, 'lon': 114.1955, 'name': '五芳街 沙田坳'},
    '20180029c69feeb': {'lat': 22.3482, 'lon': 114.1955, 'name': '五芳街 沙田坳'},
    '20080029c1e38f3': {'lat': 22.33859, 'lon': 114.19964, 'name': '五芳街'},
    '20080029c5e3677': {'lat': 22.34135, 'lon': 114.20243, 'name': 'Diamond Hill'},
    # '20180029c4574b6': {'lat': -1, 'lon': -1, 'name': '畢架山'},
    # '20080029c4574b6': {'lat': -1, 'lon': -1, 'name': '畢架山'},
    # '20080029c1ea300': {'lat': -1, 'lon': -1, 'name': 'Kowloon Bay'},
    '2005813d31be7ec': {'lat': 22.33668, 'lon': 114.19884, 'name': '八達街'},
    '20180029c7b7b60': {'lat': 22.32514, 'lon': 114.21082, 'name': 'Kowloon Bay(Metro Center 2)'},
    '20080029c69ffc2': {'lat': -22.31499, 'lon': 114.18307, 'name': '何文田邨適文樓'},
}

gt_distribution = {
    'diamond_200m': {'date': ['2022/07/06 10:58:00', '2022/07/06 11:08:00'], 'coor': [22.3412944950869, 114.20073744876093]},
    'dayoujie_500m': {'date': ['2022/07/06 11:24:00', '2022/07/06 11:34:00'], 'coor': [22.338186087878373, 114.19885639530463]},
    'huangdaxian_1000m': {'date': ['2022/07/19 10:40:00', '2022/07/19 10:50:00'], 'coor': [22.341840882178104, 114.19449022248844]},
    'fuhaodongfang_1500m': {'date': ['2022/07/06 17:10:00', '2022/07/06 17:20:00'], 'coor': [22.328988088616413, 114.1925017465669]},
    'songhuangtai_2000m': {'date': ['2022/07/06 18:16:00', '2022/07/06 18:26:00'], 'coor': [22.325466338854763, 114.19031932195354]},
}

gt_point = {
    'dayoujie': {'date': '2022/07/06 11:48:00','coor': [22.338173421484996, 114.20168251817528]},
    'caihong': {'date': '2022/07/06 11:56:00','coor': [22.336047918539958, 114.20768103936794]},
    'caiyun': {'date': '2022/07/06 11:58:00','coor': [22.333539410864162, 114.21244505687329]},
    'banyuelou': {'date': '2022/07/06 11:59:00','coor': [22.332714071623705, 114.21623798449995]},
    'shunlixiaofang': {'date': '2022/07/06 12:02:00','coor': [22.331534585833744, 114.22241512074318]},
    'goodhope': {'date': '2022/07/06 12:04:00','coor': [22.33385730934927, 114.22234763216436]},
    'andachengdao': {'date': '2022/07/06 12:06:00','coor': [22.33471046097287, 114.23013126761987]},
    'caiyihuangyuan': {'date': '2022/07/06 17:07:00','coor': [22.33492846990551, 114.20209446741167]},
    'yugangwan': {'date': '2022/07/06 17:08:00','coor': [22.332432475603923, 114.1971317522924]},
}

In [17]:
data = pd.read_csv('survey/pixel/2022-07-06_lora.csv')
data.columns = ['tag', 'router', 'rssi', 'ts']

In [18]:
songhuangtai = data[(data.ts > date2ts('2022/07/06 10:58:00'))&(data.ts < date2ts('2022/07/06 11:06:00'))]

routers = songhuangtai['router'].drop_duplicates(keep='first').tolist()

for r in routers:
    r_data = data[(data.router == r)]
    size = len(r_data)
    rssi_avg = round(np.mean(r_data.rssi),2)
    rssi_std = round(np.std(r_data.rssi), 2)
    if r[2:] in router_info.keys():
        print(r[2:], size, rssi_avg, rssi_std, router_info[r[2:]])
    # else:
    #     print(r[2:], size, rssi_avg, rssi_std)

20180029c69feeb 1076 -103.83 5.94 {'lat': 22.3482, 'lon': 114.1955, 'name': '五芳街 沙田坳'}
20080029c5e3677 483 -111.58 4.32 {'lat': 22.34135, 'lon': 114.20243, 'name': 'Diamond Hill'}
20080029c69feeb 647 -83.65 6.26 {'lat': 22.3482, 'lon': 114.1955, 'name': '五芳街 沙田坳'}
20180029c7b7b60 80 -98.68 5.31 {'lat': 22.32514, 'lon': 114.21082, 'name': 'Kowloon Bay(Metro Center 2)'}


In [19]:

def check(raw_data):
    # raw_data = data[(data.ts > date2ts('2022/07/06 18:16:00'))&(data.ts < date2ts('2022/07/06 18:16:00'))]
    routers = raw_data['router'].drop_duplicates(keep='first').tolist()

    for r in routers:
        r_data = data[(data.router == r)]
        size = len(r_data)
        rssi_avg = round(np.mean(r_data.rssi),2)
        rssi_std = round(np.std(r_data.rssi), 2)
        if r[2:] in router_info.keys():
            print(r[2:], size, rssi_avg, rssi_std, router_info[r[2:]])
        else:
            print(r[2:], size, rssi_avg, rssi_std)


for item in gt_distribution:
    s_date = gt_distribution[item]['date'][0]
    e_date = gt_distribution[item]['date'][1]
    # print(s_date, e_date)
    duration = date2ts(e_date) - date2ts(s_date)
    raw_data = data[(data.ts > date2ts(s_date))&(data.ts < date2ts(e_date))]
    print(item, raw_data.shape, duration)

    routers = raw_data['router'].drop_duplicates(keep='first').tolist()

    for r in routers:
        r_data = raw_data[(raw_data.router == r)]
        size = len(r_data)
        rssi_avg = round(np.mean(r_data.rssi),2)
        rssi_std = round(np.std(r_data.rssi), 2)
        if r[2:] in router_info.keys():
            print('-----', r[2:], size, rssi_avg, rssi_std, router_info[r[2:]])

diamond_200m (335, 4) 600.0
----- 20180029c69feeb 126 -105.41 4.5 {'lat': 22.3482, 'lon': 114.1955, 'name': '五芳街 沙田坳'}
----- 20080029c5e3677 130 -111.22 3.46 {'lat': 22.34135, 'lon': 114.20243, 'name': 'Diamond Hill'}
----- 20080029c69feeb 7 -86.43 3.11 {'lat': 22.3482, 'lon': 114.1955, 'name': '五芳街 沙田坳'}
----- 20180029c7b7b60 1 -97.0 0.0 {'lat': 22.32514, 'lon': 114.21082, 'name': 'Kowloon Bay(Metro Center 2)'}
dayoujie_500m (371, 4) 600.0
----- 20080029c69feeb 89 -83.82 4.52 {'lat': 22.3482, 'lon': 114.1955, 'name': '五芳街 沙田坳'}
----- 20180029c69feeb 127 -103.01 4.73 {'lat': 22.3482, 'lon': 114.1955, 'name': '五芳街 沙田坳'}
----- 20080029c1e38f3 102 -110.01 9.31 {'lat': 22.33859, 'lon': 114.19964, 'name': '五芳街'}
----- 20080029c5e3677 32 -112.81 1.93 {'lat': 22.34135, 'lon': 114.20243, 'name': 'Diamond Hill'}
----- 2005813d31be7ec 3 -111.0 1.63 {'lat': 22.33668, 'lon': 114.19884, 'name': '八達街'}
huangdaxian_1000m (364, 4) 600.0
----- 20080029c5e3677 13 -115.69 2.33 {'lat': 22.34135, 'lon': 11